# MediaPipe 추론 속도 체크

* 1920x1080 1000개 데이터 기준 추론 속도 체크

In [1]:
import mediapipe as mp

mpDraw = mp.solutions.drawing_utils
mpFaceMesh = mp.solutions.face_mesh
faceMesh = mpFaceMesh.FaceMesh()

In [2]:
import os

root_path = 'D:/1920x1080 faces/'
img_paths = os.listdir(root_path)
img_paths

['1920x1080 actor',
 '1920x1080 actress',
 '1920x1080 drama',
 '1920x1080 faces',
 '1920x1080 singer']

In [3]:
from glob import glob

all_imgs = []
for img_path in img_paths:
    imgs = glob(root_path+img_path+'/*.jpg')
    all_imgs.extend(imgs)
# all_imgs = extend(all_imgs)
len(all_imgs)

1000

In [4]:
all_imgs[:5]

['D:/1920x1080 faces/1920x1080 actor\\image_1.jpg',
 'D:/1920x1080 faces/1920x1080 actor\\image_10.jpg',
 'D:/1920x1080 faces/1920x1080 actor\\image_100.jpg',
 'D:/1920x1080 faces/1920x1080 actor\\image_101.jpg',
 'D:/1920x1080 faces/1920x1080 actor\\image_102.jpg']

### Detect Boxing 이미지 저장

In [5]:
import cv2
import mediapipe as mp
import time

start = time.time()

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

IMAGE_FILES = all_imgs
with mp_face_detection.FaceDetection(
    model_selection=1, min_detection_confidence=0.5) as face_detection:
  for idx, file in enumerate(IMAGE_FILES):
    image = cv2.imread(file)
    # Convert the BGR image to RGB and process it with MediaPipe Face Detection.
    results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Draw face detections of each face.
    if not results.detections:
      continue
    annotated_image = image.copy()
    for detection in results.detections:
    #   print('Nose tip:')
    #   print(mp_face_detection.get_key_point(
    #       detection, mp_face_detection.FaceKeyPoint.NOSE_TIP))
      mp_drawing.draw_detection(annotated_image, detection)
      
    cv2.imwrite('MediaPipe_Images/' + str(idx) + '.jpg', annotated_image)

print("Execute Time : ", time.time()-start)

### 안면 인식 후 Crop 이미지 저장

In [ ]:
import cv2
import mediapipe as mp
import time

start = time.time()

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

IMAGE_FILES = all_imgs
output_directory = 'MediaPipe_Images/'  # 결과 이미지 저장할 디렉토리


except_cnt = 0
people_cnt = 0

with mp_face_detection.FaceDetection(
    model_selection=1, min_detection_confidence=0.5) as face_detection:

    for idx, file in enumerate(IMAGE_FILES):
        image = cv2.imread(file)
        # Convert the BGR image to RGB and process it with MediaPipe Face Detection.
        results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # Draw face detections of each face.
        if not results.detections:
            continue

        cnt = 0
        try:
            for face_id, detection in enumerate(results.detections):
                cnt += 1
                people_cnt += 1
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = image.shape
                # Calculate the absolute coordinates of the bounding box.
                x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)

                # Crop the face region from the original image.
                face_roi = image[y:y+h, x:x+w]

                # Resize the face region to (224, 224) size.
                resized_face = cv2.resize(face_roi, (224, 224))

                # Save the resized face to the output directory.
                output_filename = f'{output_directory}{str(idx)}_face_{cnt}.jpg'
                cv2.imwrite(output_filename, resized_face)
        
        except:
            # print("except!!!")
            # print(file)
            except_cnt += 1

print("Face Recognition : MediaPipe")
print("Test Datasets : ", len(all_imgs))
print("Execute Time : ", time.time()-start)

Face Recognition : MediaPipe
Test Datasets :  1000
Execute Time :  35.97686815261841


In [ ]:
print("Except Case : ",except_cnt)
print("Detected People : ",people_cnt)

Except Case :  28
Detected People :  1170


### WEBCAM - MediaPipe 적용

In [ ]:
import cv2
import time
import mediapipe as mp
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime


start_time = time.time()


# 모델 선언
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("camera open failed")
    exit()

#capNum = 0 
capNum = 1
while True:
    
    ret, frame = cap.read()
    if not ret:
        print("Can't read camera")
        break

    # cv2.imshow('PC_camera', frame)

    now = datetime.now()
    now_path = str(now.date())+'-'+str(now.hour)+'-'+str(now.minute)+'-'+str(now.second)

    print(type(frame))
    print(frame.shape)

    if cv2.waitKey(1) == ord('q'):
        break

    # 현재 시간
    now = datetime.now()
    now_path = str(now.date())+'-'+str(now.hour)+'-'+str(now.minute)+'-'+str(now.second)

    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:

        # frame = Image.open(frame)
        # frame = np.array(frame)

        # Convert the BGR image to RGB and process it with MediaPipe Face Detection.
        results = face_detection.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        # results = face_detection.process(frame)


        # Draw face detections of each face.
        if not results.detections:
            continue

        cnt = 0
        try:
            for face_id, detection in enumerate(results.detections):
                cnt += 1
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = frame.shape
                # Calculate the absolute coordinates of the bounding box.
                x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)

                # Crop the face region from the original image.
                face_roi = image[y:y+h, x:x+w]

                # Resize the face region to (224, 224) size.
                resized_face = cv2.resize(face_roi, (224, 224))

                # Save the resized face to the output directory.
                output_filename = '/WebCAM_MediaPipe/'+str(now_path)+'/face_'+str(cnt)+'.png'
                print(output_filename)
                cv2.imwrite(output_filename, resized_face)
                cv2.imshow('face',resized_face)
        
        except:
            print("except!!!")
            print(file)
            except_cnt += 1
    
    print("Time sleep 5 seconds")
    time.sleep(5)

finish_time = time.time()
num_time = finish_time - start_time
print(num_time)
cap.release()
cv2.destroyAllWindows()

<class 'numpy.ndarray'>
(480, 640, 3)
/WebCAM_MediaPipe/2023-07-19-21-27-16/face_1.png
Time sleep 5 seconds
<class 'numpy.ndarray'>
(480, 640, 3)
/WebCAM_MediaPipe/2023-07-19-21-27-21/face_1.png
Time sleep 5 seconds


KeyboardInterrupt: 

: 

In [ ]:
import cv2
import time
from facenet_pytorch import MTCNN
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt


start_time = time.time()


# 모델 선언
mtcnn = MTCNN(image_size=224,margin=60, keep_all=True, post_process=False, device='cuda:0',)

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("camera open failed")
    exit()

#capNum = 0 
capNum = 1
while True:
    
    ret, frame = cap.read()
    if not ret:
        print("Can't read camera")
        break

    #cv2.imshow('PC_camera', frame)

    if cv2.waitKey(1) == ord('q'):
        break

    #img_captured = cv2.imwrite('save_img_file/captured_%03d.png' % capNum, frame)
    #print('%03d' % capNum + '번 사진 촬영 완료\n')
    #time.sleep(3)

     #저장된 이미지
    #image_path = 'save_img_file/captured_%03d.png' % capNum
    #capNum += 1
    #속도 확인을 위한 이미지 
    # if capNum < 201:
    #     image_path = 'D:/1920x1080 faces/1920x1080 actor/image_%d.jpg' % capNum
    # elif capNum < 401:
    #     image_path = 'D:/1920x1080 faces/1920x1080 actress/image_%d.jpg' % (capNum-200)
    # elif capNum < 601:
    #     image_path = 'D:/1920x1080 faces/1920x1080 drama/image_%d.jpg' % (capNum-400)
    # elif capNum < 801:
    #     image_path = 'D:/1920x1080 faces/1920x1080 faces/image_%d.jpg' % (capNum-600)
    # else:
    #     image_path = 'D:/1920x1080 faces/1920x1080 singer/image_%d.jpg' % (capNum-800)
    

    # frame = Image.open(image_path)
    frame = np.array(frame)

    # 이미지 확인
    #plt.figure(figsize=(12, 8))
    #plt.imshow(frame)
    #plt.axis('off')
    #plt.show()

    # 안면 검출 이미지 저장 경로
    recog_path = 'face_recognition/data/recog_images/'

    # 현재 시간
    #now = datetime.now()
    #now_path = str(now.date())+'-'+str(now.hour)+'-'+str(now.minute)+'-'+str(now.second)

    # face detect 모델 실행, 결과 저장(save_path=현재시간+n개)
    #faces = mtcnn(frame,save_path=recog_path+now_path+'/face.png')
    try:
        faces = mtcnn(frame,save_path=recog_path+'/face_%d.png' % capNum )
        
    except:
        pass
    print(capNum)
    capNum += 1
    

    # 검출된 안면 갯수
    #print("Face Detect : " + str(len(faces)))

    # Multi Label 해야하는 이미지들
    #recog_images = glob(recog_path+now_path+'/*.png')
    #print(recog_images)
    #time.sleep(10)
    if capNum == 5:
        break
finish_time = time.time()
num_time = finish_time - start_time
print(num_time)
cap.release()
cv2.destroyAllWindows()

1
2
3
4
1.6495461463928223


### WebCAM_MediaPipe/log_2023_7_20_1_26_7_face_0.png


In [ ]:
import time
import mediapipe as mp
import cv2
from datetime import datetime

start_time = time.time()

# 모델 선언
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# 웹캠 열기
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Camera open failed")
    exit()

root_save_path = 'WebCAM_MediaPipe/'

while True:
    ret, frame = cap.read()
    if not ret:
        print("Can't read camera")
        break

    # 현재 시간
    now = datetime.now()
    now_path = 'log_'+str(now.year) + '_' + str(now.month) + '_' + str(now.day) + '_' + str(now.hour) + '_' + str(now.minute) + '_' + str(now.second)

    save_path = root_save_path + now_path

    # 웹캠에서 프레임 읽기
    ret, frame = cap.read()
    if not ret:
        print("Can't read camera")
        break

    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
        # Convert the BGR image to RGB and process it with MediaPipe Face Detection.
        results = face_detection.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # Draw face detections of each face.
        if results.detections:
            cnt = 0
            cropped_images = []  # 전역 변수로 cropped_images 리스트를 선언
            for face_id, detection in enumerate(results.detections):
                cnt += 1
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = frame.shape
                # Calculate the absolute coordinates of the bounding box.
                x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)

                # Crop the face region from the original image.
                face_roi = frame[y:y + h, x:x + w]

                # Resize the face region to (224, 224) size.
                resized_face = cv2.resize(face_roi, (224, 224))
                cropped_images.append(resized_face)

    for idx, cropped_image in enumerate(cropped_images):
        resized_cropped_image = cv2.resize(cropped_image, (224, 224))
        output_filename = root_save_path + now_path +'_face_' + str(idx+1) + '.png' 
        print(output_filename)
        # f'WebCAM_MediaPipe/face_{now_path}_{idx}.png'
        # output_filename = f'WebCAM_MediaPipe/{now_path}/face_{idx}.png'
        cv2.imwrite(output_filename, resized_cropped_image)

    # 프레임에 얼굴 인식 결과 그리기
    # if results.detections:
    #     for detection in results.detections:
    #         mp_drawing.draw_detection(frame, detection)

    # cv2.imshow('PC_camera', frame)

    # if cv2.waitKey(1) == ord('q'):
    #     break

# 웹캠 해제
cap.release()
cv2.destroyAllWindows()

WebCAM_MediaPipe/log_2023_7_20_1_26_7_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_7_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_7_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_7_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_7_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_7_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_7_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_8_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_8_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_8_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_8_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_8_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_8_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_8_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_8_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_8_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_8_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_8_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_8_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_8_face_0.png
WebCAM_MediaPipe/log

WebCAM_MediaPipe/log_2023_7_20_1_26_15_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_15_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_16_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_16_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_16_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_16_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_16_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_16_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_16_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_16_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_16_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_16_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_16_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_16_face_0.png
WebCAM_MediaPipe/log_2023_7_20_1_26_16_face_0.png


### WebCAM_MediaPipe/log_2023_7_20_1_36_12/_face_1.png


In [1]:
import time
import mediapipe as mp
import cv2
from datetime import datetime
import os

start_time = time.time()

# 모델 선언
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# 웹캠 열기
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Camera open failed")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Can't read camera")
        break

    # 현재 시간
    now = datetime.now()
    now_path = 'log_'+str(now.year) + '_' + str(now.month) + '_' + str(now.day) + '_' + str(now.hour) + '_' + str(now.minute) + '_' + str(now.second) +'/'

    # 웹캠에서 프레임 읽기
    ret, frame = cap.read()
    if not ret:
        print("Can't read camera")
        break

    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
        # Convert the BGR image to RGB and process it with MediaPipe Face Detection.
        results = face_detection.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # Draw face detections of each face.
        if results.detections:
            cnt = 0
            cropped_images = []  # 전역 변수로 cropped_images 리스트를 선언
            for face_id, detection in enumerate(results.detections):
                cnt += 1
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = frame.shape
                # Calculate the absolute coordinates of the bounding box.
                x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)

                # Crop the face region from the original image.
                face_roi = frame[y:y + h, x:x + w]

                # Resize the face region to (224, 224) size.
                resized_face = cv2.resize(face_roi, (224, 224))
                cropped_images.append(resized_face)

    root_save_path = 'WebCAM_MediaPipe/'

    for idx, cropped_image in enumerate(cropped_images):
        resized_cropped_image = cv2.resize(cropped_image, (224, 224))

        isExist = os.path.exists(root_save_path+now_path)
        if not isExist:
            os.makedirs(root_save_path+now_path)
        output_filename = root_save_path+now_path + '_face_' + str(idx+1) + '.png' 
        print(output_filename)
        # f'WebCAM_MediaPipe/face_{now_path}_{idx}.png'
        # output_filename = f'WebCAM_MediaPipe/{now_path}/face_{idx}.png'
        cv2.imwrite(output_filename, resized_cropped_image)

    # 프레임에 얼굴 인식 결과 그리기
    # if results.detections:
    #     for detection in results.detections:
    #         mp_drawing.draw_detection(frame, detection)

    # cv2.imshow('PC_camera', frame)

    # if cv2.waitKey(1) == ord('q'):
    #     break

# 웹캠 해제
cap.release()
cv2.destroyAllWindows()

WebCAM_MediaPipe/log_2023_7_20_11_1_52/_face_1.png
WebCAM_MediaPipe/log_2023_7_20_11_1_52/_face_2.png
WebCAM_MediaPipe/log_2023_7_20_11_1_52/_face_1.png
WebCAM_MediaPipe/log_2023_7_20_11_1_52/_face_2.png
WebCAM_MediaPipe/log_2023_7_20_11_1_52/_face_1.png
WebCAM_MediaPipe/log_2023_7_20_11_1_52/_face_2.png
WebCAM_MediaPipe/log_2023_7_20_11_1_52/_face_1.png
WebCAM_MediaPipe/log_2023_7_20_11_1_52/_face_2.png
WebCAM_MediaPipe/log_2023_7_20_11_1_52/_face_1.png
WebCAM_MediaPipe/log_2023_7_20_11_1_52/_face_2.png
WebCAM_MediaPipe/log_2023_7_20_11_1_52/_face_1.png
WebCAM_MediaPipe/log_2023_7_20_11_1_52/_face_2.png
WebCAM_MediaPipe/log_2023_7_20_11_1_52/_face_1.png
WebCAM_MediaPipe/log_2023_7_20_11_1_52/_face_2.png
WebCAM_MediaPipe/log_2023_7_20_11_1_52/_face_1.png
WebCAM_MediaPipe/log_2023_7_20_11_1_52/_face_2.png
WebCAM_MediaPipe/log_2023_7_20_11_1_53/_face_1.png
WebCAM_MediaPipe/log_2023_7_20_11_1_53/_face_2.png
WebCAM_MediaPipe/log_2023_7_20_11_1_53/_face_1.png
WebCAM_MediaPipe/log_2023_7_20_

WebCAM_MediaPipe/log_2023_7_20_11_1_59/_face_1.png
WebCAM_MediaPipe/log_2023_7_20_11_1_59/_face_2.png
WebCAM_MediaPipe/log_2023_7_20_11_1_59/_face_1.png
WebCAM_MediaPipe/log_2023_7_20_11_1_59/_face_2.png
WebCAM_MediaPipe/log_2023_7_20_11_1_59/_face_1.png
WebCAM_MediaPipe/log_2023_7_20_11_1_59/_face_2.png
WebCAM_MediaPipe/log_2023_7_20_11_1_59/_face_1.png
WebCAM_MediaPipe/log_2023_7_20_11_1_59/_face_2.png
WebCAM_MediaPipe/log_2023_7_20_11_2_0/_face_1.png
WebCAM_MediaPipe/log_2023_7_20_11_2_0/_face_2.png
WebCAM_MediaPipe/log_2023_7_20_11_2_0/_face_1.png
WebCAM_MediaPipe/log_2023_7_20_11_2_0/_face_2.png
WebCAM_MediaPipe/log_2023_7_20_11_2_0/_face_1.png
WebCAM_MediaPipe/log_2023_7_20_11_2_0/_face_2.png
WebCAM_MediaPipe/log_2023_7_20_11_2_0/_face_1.png
WebCAM_MediaPipe/log_2023_7_20_11_2_0/_face_2.png
WebCAM_MediaPipe/log_2023_7_20_11_2_0/_face_1.png
WebCAM_MediaPipe/log_2023_7_20_11_2_0/_face_2.png
WebCAM_MediaPipe/log_2023_7_20_11_2_0/_face_1.png
WebCAM_MediaPipe/log_2023_7_20_11_2_0/_fac